# 論文修正用、D2NNの学習

In [1]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/D2NN.git

import sys
sys.path.append('/content/D2NN')

from google.colab import drive
drive.mount('/content/drive')

Cloning into 'D2NN'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 87 (delta 30), reused 74 (delta 17), pack-reused 0
Unpacking objects: 100% (87/87), done.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import time
import os
import json
import sys

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from module.two_dim.layers import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.2
Python: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
#@title デフォルトのタイトル テキスト
wavelength = 633.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}

In [5]:
def create_model(**kwargs):
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    phi_max = kwargs["phi"]
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization(0.5, 0.0, 1.0)(x)
    x = IntensityToElectricField()(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = ElectricFieldToIntensity(normalization="max")(x)
    x = MNISTDetector(10)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [6]:
phi_list = [2.0*np.pi/100, 2*np.pi/10, 2*np.pi/5, 2*np.pi/3, 2*np.pi/2, 2*np.pi]
for i, phi in enumerate(phi_list):
  model = create_model(phi=phi)
  model.summary()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=['accuracy']
      )

  epochs = 50
  early_stopping = tf.keras.callbacks.EarlyStopping(
      monitor='accuracy',
      min_delta=0.05,
      patience=2,
  )

  model_name = "20220810_" + str(i + 1)
  cholab_path = "/content/drive/MyDrive/D2NN/"
  checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  # チェックポイントコールバックを作る
  cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

  result = model.fit(x_train,
                    y_train,
                    batch_size=64,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[cp_callback]
                    )

  path = cholab_path + "trained_model/"+ model_name
  #path = 'trained_model/test'
  model.save(path)

  df = pd.DataFrame(result.history)
  df.to_csv(path + "/history.csv")

  df = pd.DataFrame(result.history)
  df.to_csv(path + "/history.csv")

  with open(path + "/config.json", 'w') as f:
      json.dump(model.get_config(), f, indent=4)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing (ImageResizi  (None, 100, 100)         0         
 ng)                                                             
                                                                 
 image_binarization (ImageBi  (None, 100, 100)         0         
 narization)                                                     
                                                                 
 intensity_to_electric_field  (None, 100, 100)         0         
  (IntensityToElectricField)                                     
                                                                 
 modulation (Modulation)     (None, 100, 100)          10000     
                                                             

INFO:tensorflow:Assets written to: /content/drive/MyDrive/D2NN/trained_model/20220810_1/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/D2NN/trained_model/20220810_1/assets


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 100, 100)         0         
 zing)                                                           
                                                                 
 image_binarization_1 (Image  (None, 100, 100)         0         
 Binarization)                                                   
                                                                 
 intensity_to_electric_field  (None, 100, 100)         0         
 _1 (IntensityToElectricFiel                                     
 d)                                                              
                                                                 
 modulation_5 (Modulation)   (None, 100, 100)          1000

INFO:tensorflow:Assets written to: /content/drive/MyDrive/D2NN/trained_model/20220810_2/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/D2NN/trained_model/20220810_5/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/D2NN/trained_model/20220810_5/assets


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_5 (ImageResi  (None, 100, 100)         0         
 zing)                                                           
                                                                 
 image_binarization_5 (Image  (None, 100, 100)         0         
 Binarization)                                                   
                                                                 
 intensity_to_electric_field  (None, 100, 100)         0         
 _5 (IntensityToElectricFiel                                     
 d)                                                              
                                                                 
 modulation_25 (Modulation)  (None, 100, 100)          1000

INFO:tensorflow:Assets written to: /content/drive/MyDrive/D2NN/trained_model/20220810_6/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/D2NN/trained_model/20220810_6/assets
